In [1]:
import subprocess

In [4]:
def parse_file(filename, train_file, validation_file, test_file, train_samples, tv_samples, ratio=1):
    with open(filename) as bot:
        for _ in range(int(train_samples / 2 / ratio)):
            train_file.write(bot.readline())

        # Test + Validation Samples
        for _ in range(int(tv_samples / 2 / ratio)):
            validation_file.write(bot.readline())
            test_file.write(bot.readline())

In [5]:
def shuffle_output(filename):
    cmd = subprocess.run(
        [
            'shuf ' + filename + '-temp.txt' + ' > ' + filename + '.txt' +
            ' && rm ' + filename + '-temp.txt' + 
            ' && wc -l ' + filename + '.txt'
        ],
        shell=True,
        stdout=subprocess.PIPE)
    print(cmd.stdout.decode("utf-8").strip())

In [6]:
def generate_dataset(total_size, train_samples, tv_samples, dataset=3, combined=False):
    print('train      samples:', train_samples)
    print('validation samples:', tv_samples)
    print('test       samples:', tv_samples)

    train_bot      = open('train-bot-temp.txt'     , 'w')
    train_gen      = open('train-gen-temp.txt'     , 'w')
    test_bot       = open('test-bot-temp.txt'      , 'w')
    test_gen       = open('test-gen-temp.txt'      , 'w')
    validation_bot = open('validation-bot-temp.txt', 'w')
    validation_gen = open('validation-gen-temp.txt', 'w')

    ratio = 3 if combined else 1

    if combined or dataset == 1:
        parse_file('input-bot-1.txt',
                   train_file=train_bot,
                   validation_file=validation_bot,
                   test_file=test_bot,
                   train_samples=train_samples,
                   tv_samples=tv_samples,
                   ratio=ratio)

    if combined or dataset == 2:
        parse_file('input-bot-2.txt',
                   train_bot,
                   validation_bot,
                   test_bot,
                   train_samples,
                   tv_samples=tv_samples,
                   ratio=ratio)

    if combined or dataset == 3:
        parse_file('input-bot-3.txt',
                   train_bot,
                   validation_bot,
                   test_bot,
                   train_samples,
                   tv_samples=tv_samples,
                   ratio=ratio)

    parse_file('input-gen.txt',
               train_file=train_gen,
               validation_file=validation_gen,
               test_file=test_gen,
               train_samples=train_samples,
               tv_samples=tv_samples)

    train_bot.close()
    validation_bot.close()
    test_bot.close()
    train_gen.close()
    validation_gen.close()
    test_gen.close()

    shuffle_output('train-bot')
    shuffle_output('train-gen')
    shuffle_output('validation-bot')
    shuffle_output('validation-gen')
    shuffle_output('test-bot')
    shuffle_output('test-gen')
    cmd = subprocess.run(
        ['cat train-bot.txt validation-bot.txt > trv-bot.txt'],
        shell=True,
        stdout=subprocess.PIPE)
    if cmd.returncode != 0:
        print(cmd.stdout.decode("utf-8").strip())
    cmd = subprocess.run(
        ['cat train-gen.txt validation-gen.txt > trv-gen.txt'],
        shell=True,
        stdout=subprocess.PIPE)
    if cmd.returncode != 0:
        print(cmd.stdout.decode("utf-8").strip())

In [7]:
total_size = 800000
train_samples = int(total_size / 10 * 6)
tv_samples = int(total_size / 5)

In [8]:
generate_dataset(total_size, train_samples, tv_samples, dataset=3, combined=False)

train      samples: 480000
validation samples: 160000
test       samples: 160000
240000 train-bot.txt
240000 train-gen.txt
80000 validation-bot.txt
80000 validation-gen.txt
80000 test-bot.txt
80000 test-gen.txt
